In [42]:
import finlab
from finlab import data
import pandas as pd
import datetime as dt
import numpy as np
import math
from dotenv import load_dotenv
import os
import time
from finlab.backtest import sim
import scipy.stats as stats

def config():
    load_dotenv()


    
config()

finlab.login(str(os.getenv('FINLAB_API')))

輸入成功!


In [123]:
# with data.universe(market='TSE_OTC',category='金融'): # 上市+上櫃
#     financialColumns =data.get('etl:adj_close').columns
# with data.universe(market='TSE_OTC',category='金融'): # 上市+上櫃
#     financialColumns =data.get('etl:adj_close').columns
    
# with data.universe(market='TSE_OTC'):
#     allColumns =data.get('etl:adj_close').columns

with data.universe(market='TSE',category='金融'): # 上市 (ETF母體)
    financialTseColumns =data.get('etl:adj_close').columns
with data.universe(market='other_securities',category='其他證券'): # 上市 (ETF or other 特別股)
    ETF_Columns =data.get('etl:adj_close').columns
with data.universe(market='TSE'):
    allTseColumns =data.get('etl:adj_close').columns

# # 上櫃
# with data.universe(market='OTC',category='金融'):
#     financialOtcColumns =data.get('etl:adj_close').columns
# with data.universe(market='OTC'):
#     allOtcColumns =data.get('etl:adj_close').columns

data.set_universe(market='TSE', category = ["光電業", "其他", "其他電子業", "化學工業", "半導體", "塑膠工業", "存託憑證", 
                                            "建材營造", "文化創意業", 
                                            "橡膠工業", "水泥工業", "汽車工業", "油電燃氣業", "玻璃陶瓷", "生技醫療", 
                                            "生技醫療業", "紡織纖維", 
                                            "航運業", "觀光事業", "貿易百貨", "資訊服務業", "農業科技", "通信網路業", 
                                            "造紙工業", "金融", 
                                            "鋼鐵工業", "電器電纜", "電子商務", "電子通路業", "電子零組件", 
                                            "電機機械", "電腦及週邊", "食品工業"]) # set data.universe and exclude EFT



def Z(df):
    df= df.apply(stats.zscore,axis=1,nan_policy='omit') # zscore, 忽略nan
    cond = (df>3) | (df<-3) # 只留下3個std以內的資料,離群值不採納
    df[cond] = np.nan
    return df

def diffList(a,b): #在A裡面不在B裡面的元素
    return [i for i in a if i not in b]

In [127]:
# security_categories = data.get('security_categories')
# security_categories['category'].unique()

# #[security_categories['market']== 'TSE']
# #security_categories['category']== '其他' & 

array(['金融業', '其他證券', 'nan', '水泥工業', '食品工業', '農業科技', '觀光事業', '其他',
       '生技醫療業', '塑膠工業', '建材營造', '汽車工業', '電子零組件', '電子零組件業', '紡織纖維', '貿易百貨',
       '電機機械', '電腦及週邊設備業', '電器電纜', '化學工業', '生技醫療', '其他電子業', '玻璃陶瓷',
       '造紙工業', '鋼鐵工業', '橡膠工業', '航運業', '半導體業', '半導體', '通信網路業', '電腦及週邊',
       '光電業', '電子通路業', '資訊服務業', '油電燃氣業', '金融保險業', '文化創意業', '電子商務', '電子工業',
       '存託憑證'], dtype=object)

In [134]:

收盤價 = data.get('etl:adj_close')# Capital Stock Value
股本 = data.get('financial_statement:股本').index_str_to_date()

# Cash Flow per Share
每股營業現金流量 = data.get('fundamental_features:每股現金流量').index_str_to_date()

# EBITDA to Enterprise Value Ratio
EBITDA = data.get('fundamental_features:EBITDA').index_str_to_date()
負債總額 = data.get('financial_statement:負債總額').index_str_to_date() 
現金及約當現金 = data.get('financial_statement:期末現金及約當現金餘額').index_str_to_date()
市值 = 股本 / 10 * 收盤價
企業價值 = 市值 + 負債總額 - 現金及約當現金
EBITDA對企業價值比 = EBITDA / 企業價值

# Pause to prevent connection issues
time.sleep(5)

# Earnings to Price Ratio
每股盈餘 = data.get('financial_statement:每股盈餘').index_str_to_date()
益本比 = 每股盈餘 / 收盤價

# Shareholder Yield
股息殖利率 = data.get('price_earning_ratio:殖利率(%)').index_str_to_date()
發行股數 = data.get('internal_equity_changes:發行股數').index_str_to_date() # Modified from 股本
前一年發行股數 = data.get('internal_equity_changes:發行股數').shift(12).index_str_to_date()  # 12 months for a year
庫藏股回購率 = (發行股數 - 前一年發行股數) * (收盤價 / 市值)
股東收益率 = 股息殖利率 + 庫藏股回購率

# 計算價值因子分數

價值因子分數 = 0.4 * Z(每股營業現金流量) + 0.4* Z(益本比) + 0.2 * Z(股東收益率) #用非金融股算法計算全部
價值因子分數_金融 = 0.3 * Z(每股營業現金流量) +0.3 * Z(EBITDA對企業價值比) + 0.3 * Z(益本比) + 0.1 * Z(股東收益率) #用金融股算法計算全部
finCol = 價值因子分數_金融.columns.intersection(financialTseColumns) # 取交集過濾掉已經下市的股票
價值因子分數[finCol] = 價值因子分數_金融[finCol] # 再用金融股的值代入金融股的值
# 價值因子分數 # 所有TSE & OTC 的股票價值因子分數
for column in ETF_Columns:
    if column in 價值因子分數.columns:
        價值因子分數 = 價值因子分數.drop(column, axis=1)
價值因子分數

symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-28,0.006135,0.029653,-0.006109,-0.065086,-0.045076,0.141722,-0.133233,0.102344,0.021052,0.179716,...,0.574900,0.124084,-0.977850,0.062614,0.087394,0.059013,0.147339,-0.194768,-0.202801,0.128061
2020-12-29,0.006924,0.031048,-0.003064,-0.062726,-0.043282,0.144786,-0.134295,0.103030,0.021708,0.180813,...,0.577073,0.124467,-0.977550,0.062522,0.087551,0.059698,0.150548,-0.195521,-0.201941,0.128429
2020-12-30,0.006787,0.031694,-0.003615,-0.062546,-0.042979,0.145828,-0.134345,0.104271,0.022143,0.181486,...,0.576833,0.124320,-0.977336,0.062335,0.087754,0.059713,0.150494,-0.195602,-0.201735,0.128603


In [68]:
EBITDA.isnull().sum()

symbol
000116    966
000930    966
000960    966
0009A0    966
1101       17
         ... 
9951       15
9955       16
9958       16
9960       16
9962       16
Length: 1888, dtype: int64

Index(['2013-Q1', '2013-Q2', '2013-Q3', '2013-Q4', '2014-Q1', '2014-Q2',
       '2014-Q3', '2014-Q4', '2015-Q1', '2015-Q2', '2015-Q3', '2015-Q4',
       '2016-Q1', '2016-Q2', '2016-Q3', '2016-Q4', '2017-Q1', '2017-Q2',
       '2017-Q3', '2017-Q4', '2018-Q1', '2018-Q2', '2018-Q3', '2018-Q4',
       '2019-Q1', '2019-Q2', '2019-Q3', '2019-Q4', '2020-Q1', '2020-Q2',
       '2020-Q3'],
      dtype='object', name='date')

In [70]:
# 計算價值因子分數

價值因子分數 = 0.4 * Z(每股營業現金流量) + 0.4* Z(益本比) + 0.2 * Z(股東收益率) #用非金融股算法計算全部
價值因子分數_金融 = 0.3 * Z(每股營業現金流量) +0.3 * Z(EBITDA對企業價值比) + 0.3 * Z(益本比) + 0.1 * Z(股東收益率) #用金融股算法計算全部
# finCol = 價值因子分數_金融.columns.intersection(financialColumns) # 取交集過濾掉已經下市的股票
# 價值因子分數[finCol] = 價值因子分數_金融[finCol] # 再用金融股的值代入金融股的值
# 價值因子分數 # 所有TSE & OTC 的股票價值因子分數
價值因子分數.isnull().sum()

symbol
1101    208
1102    206
1103    208
1104    206
1108    206
       ... 
9951    206
9955    206
9958    206
9960    310
9962    214
Length: 1792, dtype: int64

In [147]:
data.get('fundamental_features:總負債除總淨值')

Daily usage: 1.3 / 500 MB - fundamental_features:總負債除總淨值


symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-Q1,NaN,NaN,NaN,NaN,87.688868,97.268963,76.731957,19.340632,100.204638,36.669659,...,120.545376,69.522970,738.164508,36.217489,26.273520,70.012248,133.882052,187.303406,43.132708,171.138350
2013-Q2,NaN,NaN,NaN,NaN,96.573538,117.368865,77.892512,24.405350,101.028500,39.891165,...,136.063976,90.872684,798.378280,52.718437,44.681161,72.913374,223.759407,205.358706,57.611310,163.499881
2013-Q3,NaN,NaN,NaN,NaN,87.327663,103.083446,69.939717,22.182749,98.124176,35.626198,...,130.899519,88.160378,782.687633,37.846154,29.365240,71.284718,162.623580,155.788726,46.360939,134.066127
2013-Q4,NaN,NaN,NaN,NaN,84.014684,101.238459,55.243241,22.063423,94.917546,37.366489,...,131.709663,72.421040,718.274645,39.268302,26.414704,75.928368,73.498675,170.176493,45.149029,107.773103
2014-Q1,NaN,NaN,NaN,NaN,80.570685,67.074396,58.504345,21.140627,88.334238,35.659327,...,134.593117,65.907364,713.258341,35.204850,24.022213,68.278680,73.468943,185.013641,40.560184,109.284199
2014-Q2,NaN,NaN,NaN,NaN,90.251178,79.192719,78.121902,25.571081,91.451596,44.830501,...,158.002069,97.135290,842.810728,58.379125,49.039499,70.444833,91.273933,255.820305,51.225823,166.580440
2014-Q3,NaN,NaN,NaN,NaN,84.225942,73.494134,77.189409,22.365750,86.627808,31.918668,...,156.995641,80.755210,815.742776,37.573917,24.998098,71.155305,95.927114,265.958503,61.153857,182.897960
2014-Q4,NaN,NaN,NaN,NaN,80.686349,74.874115,74.600030,22.691434,81.215004,32.015860,...,150.201304,87.113303,781.961014,45.477392,23.682991,71.946720,80.723222,273.367670,59.429376,178.178410
2015-Q1,NaN,NaN,NaN,NaN,78.636299,71.989989,74.691069,22.390809,72.016479,29.359628,...,153.548634,89.270338,775.234127,42.385229,20.304136,76.560441,144.067673,269.553511,61.487794,175.740712


In [148]:
import time

# 股東權益報酬率 (ROE) 1004 rows × 1045 columns
ROE = data.get('fundamental_features:ROE稅後').index_str_to_date()

# 收益變動率

#營業利益成長率 999 rows × 1025 columns
營業利益成長率 = data.get('fundamental_features:營業利益成長率').index_str_to_date()


# 收益變動率 999 rows × 1025 columns
收益變動率 = -(營業利益成長率.rolling(20).std())



# III. 現金流量對資產比 1004 rows × 1045 columns
營業現金流量 = data.get('fundamental_features:營運現金流')
資產總額 = data.get('financial_statement:資產總額').index_str_to_date()
現金流量對資產比 = 營業現金流量 / 資產總額

# IV. 資產報酬率 (ROA)
ROA = data.get('fundamental_features:ROA稅後息前').index_str_to_date()

# V. 毛利對資產比  1004 rows × 1041 columns
營業毛利 = data.get('financial_statement:營業毛利').index_str_to_date()
毛利對資產比 = 營業毛利 / 資產總額

#replace 槓桿度 because of 槓桿度 is lack of data
總負債除總淨值= data.get('fundamental_features:總負債除總淨值').index_str_to_date()
槓桿度 = 總負債除總淨值

# # VI. 槓桿度  2795 rows × 53 columns 

# # 長期負債 899 rows × 989 columns
# 長期負債 = data.get('financial_statement:非流動負債').index_str_to_date()
# # market_value 2801 rows × 1024 columns
# market_value = data.get('etl:market_value').index_str_to_date()

# #特別股股本 401 rows × 68 columns  !! problem found !!
# 特別股股本 = data.get('financial_statement:特別股股本').index_str_to_date()
# #市場槓桿 2795 rows × 53 columns  !! problem found !!
# 市場槓桿 = ((market_value + 長期負債 + 特別股股本) / market_value)
# 帳面價值槓桿 = (data.get('financial_statement:股東權益總額').index_str_to_date() + 長期負債 + data.get('financial_statement:特別股股本').index_str_to_date()) / data.get('financial_statement:股東權益總額').index_str_to_date()
# 負債比率 = data.get('fundamental_features:負債比率').index_str_to_date()

# # Calculating 槓桿度 (Leverage)
# 槓桿度 = -(市場槓桿 + 帳面價值槓桿 + 負債比率) / 3

# Standardizing the values using Z function
金融股品質因子分數 = (1 / 3) * (Z(ROE) + Z(收益變動率) + Z(ROA))
品質因子分數 = 0.2 * (Z(ROE) + Z(收益變動率) + Z(現金流量對資產比) + Z(毛利對資產比) + Z(槓桿度))

finCol = 金融股品質因子分數.columns.intersection(financialTseColumns) # 取交集過濾掉已經下市的股票
品質因子分數[finCol] = 金融股品質因子分數[finCol] # 再用金融股的值代入金融股的值
品質因子分數 # 所有TSE & OTC 的股票價值因子分數
品質因子分數.isnull().sum()
品質因子分數.info()


for column in ETF_Columns:
    if column in 品質因子分數.columns:
        品質因子分數 = 品質因子分數.drop(column, axis=1)
品質因子分數

<class 'finlab.dataframe.FinlabDataFrame'>
DatetimeIndex: 1004 entries, 2013-04-19 to 2021-03-22
Columns: 1025 entries, 1101 to 9958
dtypes: float64(1025)
memory usage: 7.9 MB


symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,-0.127977,-0.170040,-0.341017,-0.277124,-0.101904,0.151643,-0.312204,0.382390,0.095488,0.225968,...,0.179458,0.184262,0.105892,0.091916,0.202370,-0.060733,-0.308678,-0.378311,-0.410858,0.037898
2020-11-26,-0.128144,-0.170211,-0.341195,-0.277303,-0.102082,0.151481,-0.312367,0.382246,0.095298,0.225807,...,0.179309,0.184294,0.105773,0.091756,0.202215,-0.060742,-0.308860,-0.378488,-0.411419,0.038063
2020-11-27,-0.128447,-0.170522,-0.341534,-0.277652,-0.102387,0.151187,-0.312709,0.382062,0.094987,0.225552,...,0.179089,0.184314,0.105893,0.091463,0.201943,-0.060908,-0.309199,-0.378824,-0.412045,0.038061


symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-20,11.52095,5.871978,-259.989224,279.753666,280.854514,-16.912298,-187.055775,-6.902867,18.082024,22.087155,...,7.63424,77.559725,-11.190437,-0.261704,-13.747238,919.099456,-7.706561,-89.94325,347.554585,516.826387
2020-11-25,11.52095,5.871978,-259.989224,279.753666,280.854514,-16.912298,-187.055775,-6.902867,18.082024,22.087155,...,7.63424,77.559725,-11.190437,-0.261704,-13.747238,919.099456,-7.706561,-89.94325,347.554585,516.826387
2020-11-26,11.52095,5.871978,-259.989224,279.753666,280.854514,-16.912298,-187.055775,-6.902867,18.082024,22.087155,...,7.63424,77.559725,-11.190437,-0.261704,-13.747238,919.099456,-7.706561,-89.94325,347.554585,516.826387


In [78]:
市場槓桿.isnull().sum()

symbol
1101    1592
1220      79
1233      14
1312      13
1336     795
        ... 
8467     981
8916    1562
9906     533
9925    1818
9941    1558
Length: 111, dtype: int64

In [77]:
槓桿度.isnull().sum()

symbol
1101    1592
1220      79
1233      14
1312      13
1336     795
        ... 
8467     981
8916    1562
9906     533
9925    1818
9941    1558
Length: 111, dtype: int64

In [75]:
毛利對資產比

symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-26,0.027816,0.021196,-0.000486,0.009512,0.022575,0.054849,0.00194,0.087982,0.057636,0.066483,...,0.029613,0.005271,0.005325,0.052814,0.004806,0.070428,0.011538,0.041052,0.060391,0.017432
2020-11-27,0.027816,0.021196,-0.000486,0.009512,0.022575,0.054849,0.00194,0.087982,0.057636,0.066483,...,0.029613,0.005271,0.005325,0.052814,0.004806,0.070428,0.011538,0.041052,0.060391,0.017432
2020-11-30,0.027816,0.021196,-0.000486,0.009512,0.022575,0.054849,0.00194,0.087982,0.057636,0.066483,...,0.029613,0.005271,0.005325,0.052814,0.004806,0.070428,0.011538,0.041052,0.060391,0.017432


In [74]:
ROA

symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-26,0.418322,0.438312,0.615711,0.464,2.345613,2.124012,1.794464,1.7296,1.353575,3.538284,...,1.003004,2.488607,0.329536,-1.218704,-1.463868,2.992517,0.811042,2.80845,2.220852,0.346815
2020-11-27,0.418322,0.438312,0.615711,0.464,2.345613,2.124012,1.794464,1.7296,1.353575,3.538284,...,1.003004,2.488607,0.329536,-1.218704,-1.463868,2.992517,0.811042,2.80845,2.220852,0.346815
2020-11-30,0.418322,0.438312,0.615711,0.464,2.345613,2.124012,1.794464,1.7296,1.353575,3.538284,...,1.003004,2.488607,0.329536,-1.218704,-1.463868,2.992517,0.811042,2.80845,2.220852,0.346815


In [73]:
現金流量對資產比

symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9944,9945,9946,9949,9950,9951,9955,9958,9960,9962
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-04-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-26,-0.022954,-0.035723,-0.04753,0.006903,0.004804,0.018773,-0.003098,0.027305,0.02694,0.034936,...,0.027472,-0.001714,-0.016996,0.00464,-0.013552,0.041253,-0.02887,0.025995,0.046417,0.13864
2020-11-27,-0.022954,-0.035723,-0.04753,0.006903,0.004804,0.018773,-0.003098,0.027305,0.02694,0.034936,...,0.027472,-0.001714,-0.016996,0.00464,-0.013552,0.041253,-0.02887,0.025995,0.046417,0.13864
2020-11-30,-0.022954,-0.035723,-0.04753,0.006903,0.004804,0.018773,-0.003098,0.027305,0.02694,0.034936,...,0.027472,-0.001714,-0.016996,0.00464,-0.013552,0.041253,-0.02887,0.025995,0.046417,0.13864


AttributeError: 'DataFrame' object has no attribute 'unique'

In [165]:
income_increasing_trend

symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-04-24,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-04-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-04-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-04-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
2020-11-26,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-11-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [164]:
income_increasing_trend.nunique()

symbol
000116    1
000930    1
000960    1
0009A0    1
1101      2
         ..
9944      2
9945      2
9946      2
9955      2
9958      2
Length: 1043, dtype: int64

In [192]:
#動能因子
營業收入淨額 = data.get('financial_statement:營業收入淨額').index_str_to_date()


def keep_increasing_n_month(data, m=4, n=5, weight = 1):


    data = data.rolling(m).mean()
    # Initialize an empty list for buy signals
    zero_dataframe = pd.DataFrame(0, index=range(len(data.index)), columns=range(len(data.columns)))

    # Iterate over the columns to find increasing trends
    for column in range(len(data.columns)):

        column_data = data.iloc[:,column]
        column_zero_dataframe = zero_dataframe.iloc[:,column]
        # Iterate over the dates to find increasing trends
        for i in range(n, len(column_data)):
            # ex: look for data[3-1] see if data[3-1] > data[2-0]
            if all(column_data[j] > column_data[j - 1] for j in range(i - n + 1, i + 1)):
                column_zero_dataframe.iloc[i] = weight
            elif all(column_data[j] > column_data[j - 1] for j in range(i - n + 1, i + 1)):
                column_zero_dataframe.iloc[i] = -weight
            else:
                pass
                
    
    zero_dataframe.index = data.index
    zero_dataframe.columns = data.columns
    return zero_dataframe

income_increasing_trend = keep_increasing_n_month(營業收入淨額, n=3, m=5, weight=1)

income_increasing_trend


經常稅後淨利 = 	data.get('fundamental_features:經常稅後淨利').index_str_to_date()
稅後淨利成長動能 =  keep_increasing_n_month(經常稅後淨利, n=3, m=5, weight=1)


動能因子分數 =  (Z(稅後淨利成長動能) + Z(income_increasing_trend))



動能因子分數 # 所有TSE & OTC 的股票價值因子分數
稅後淨利成長動能.nunique()
for column in ETF_Columns:
    if column in 價值因子分數.columns:
        動能因子分數 = 動能因子分數.drop(column, axis=1)
動能因子分數['1102'].unique()




array([        nan, -0.07479589, -0.11573002, -0.13877518, -0.15209383,
       -0.14543613, -0.17545541, -0.24910356, -0.35536985, -0.50177057,
       -0.45929615, -0.37131324, -0.54069983, -1.07770849,  2.4233433 ,
        2.79399028,  4.2167687 , -0.06194297, -0.13142166, -0.21088126,
       -0.22039279, -0.18601325, -0.06194297, -0.08764263, -0.12406457,
       -0.13141809, -0.16435934, -0.26097216, -0.27579878, -0.26866883,
       -0.17579235, -0.20643406, -0.2572744 , -0.39663189, -0.37254893,
       -0.48293819, -0.64015032, -1.19937885, -1.46396255, -1.27055256,
       -0.724332  , -0.10739143, -0.09751941, -0.09301126, -0.11321509,
       -0.15157038, -0.08467267, -0.16266645, -0.14421686, -0.15728828,
       -0.1058492 , -0.1230795 , -0.16931572, -0.18117052, -0.21553235,
       -0.24139894, -0.23738076, -0.25640827, -0.29285891, -0.34112096,
       -0.38076016, -0.45558265, -0.63608615, -0.91245147, -0.84408571,
       -0.66275525,  4.76611873,  4.85013183,  4.8584005 , -0.07

In [189]:
動能因子分數

symbol,000116,000930,000960,0009A0,1101,1102,1103,1104,1108,1109,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-04-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0
2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,6.0,0.0
2020-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
經常稅後淨利.rolling(4).mean().plot()

In [180]:
動能因子分數['1101'].unique()

array([nan])

In [170]:
動能因子分數.nunique()

symbol
1101    0
1102    0
1103    0
1104    0
1108    0
       ..
9944    0
9945    0
9946    0
9955    0
9958    0
Length: 1023, dtype: int64

In [168]:
稅後淨利成長動能.nunique()

symbol
1101    2
1102    1
1103    2
1104    1
1108    2
       ..
9944    1
9945    2
9946    2
9955    1
9958    1
Length: 1025, dtype: int64

In [32]:
# 盈餘_sorted= data.get('financial_statement:每股盈餘')
# import pandas as pd


# # You will need the stock's closing price on each data cut-off date
# # Replace this list with your actual closing prices for each quarter
# closing_stock_prices = data.get('etl:adj_close')

# # Check if there are enough rows and closing prices
# if len(盈餘_sorted) >= 2 and len(closing_stock_prices) == len(盈餘_sorted):
#     # Initialize an empty list to store profit momentum values
#     profit_momentums = []

#     # Iterate through the DataFrame
#     for i in range(len(盈餘_sorted) - 1):
#         current_eps = 盈餘_sorted.iloc[i]['EPS']
#         previous_eps = 盈餘_sorted.iloc[i + 1]['EPS']
#         closing_price = closing_stock_prices[i]

#         # Calculate Profit Momentum and append to the list
#         profit_momentum = (current_eps - previous_eps) / closing_price
#         profit_momentums.append(profit_momentum)

#     # Create a new DataFrame for Profit Momentum
#     profit_momentum_df = pd.DataFrame({
#         'date': 盈餘_sorted['date'].iloc[:-1].values,  # Exclude the last date as it has no previous quarter to compare
#         '盈利動能': profit_momentums
#     })

#     print(profit_momentum_df)
# else:
#     print("Error: Not enough data or closing stock prices for calculation.")






Error: Not enough data or closing stock prices for calculation.


symbol,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,...,9939,9940,9941,9942,9943,9944,9945,9946,9955,9958
date,,,,,,,,,,,,,,,,,,,,,
2013-04-19,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-22,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-23,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-24,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2013-04-25,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-28,False,False,False,False,False,True,False,True,False,True,...,True,True,False,False,True,False,False,False,False,False
2020-12-29,False,False,False,False,False,True,False,True,False,True,...,True,True,False,False,True,False,False,False,False,False
2020-12-30,False,False,False,False,False,True,False,True,False,True,...,True,True,False,False,True,False,False,False,False,False


In [195]:
多因子分數 = 0.4 * 品質因子分數 + 0.3 * 價值因子分數 + 0.3 * 動能因子分數
多因子權重係數 = 多因子分數.applymap(lambda z: 1+z if(z>=0) else (1-z)**-1)
quantile_75 = 多因子權重係數.transpose().quantile(0.75)
position = 多因子權重係數 > quantile_75
position
report =sim(position=position, resample='Q', trade_at_price='open',position_limit=1/5, fee_ratio=1.425/1000*0.3),# stop_loss=0.3655)
report.display_mae_mfe_analysis()

,annualized_rate_of_return,sharpe,max_drawdown,win_ratio
,9.98%,0.57,-32.24%,50.68%


,entry_date,exit_date,entry_sig_date,exit_sig_date,position,period,entry_index,exit_index,return,trade_price@entry_date,...,警示股@exit_date,處置股@exit_date,全額交割股@exit_date,mae,gmfe,bmfe,mdd,pdays,weight,next_weights
stock_id,,,,,,,,,,,,,,,,,,,,,
1101 台泥,2020-10-05,NaT,2020-09-30,2021-03-31,0.004525,63.0,1899.0,-1.0,0.039518,41.50,...,False,False,False,-0.030120,0.057831,0.000000,-0.030120,37.0,0.004525,0.000000
1109 信大,2019-04-01,NaT,2019-03-29,NaT,0.004762,433.0,1529.0,-1.0,0.463986,15.10,...,False,False,False,-0.046358,0.567169,0.465016,-0.349057,427.0,0.004525,0.004484
1201 味全,2020-10-05,NaT,2020-09-30,NaT,0.004525,63.0,1899.0,-1.0,0.123371,19.65,...,False,False,False,0.000000,0.150127,0.000000,-0.043880,62.0,0.004525,0.004484
1210 大成,2020-04-01,NaT,2020-03-31,NaT,0.005051,189.0,1773.0,-1.0,0.419579,37.00,...,False,False,False,0.000000,0.463052,0.000000,-0.114270,188.0,0.004525,0.004484
1215 卜蜂,2020-10-05,NaT,2020-09-30,2021-03-31,0.004525,63.0,1899.0,-1.0,0.093889,66.50,...,False,False,False,0.000000,0.111278,0.000000,-0.032967,62.0,0.004525,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9921 巨大,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.004484
9924 福興,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.004484
9934 成霖,NaT,NaT,2021-03-31,NaT,NaN,NaN,NaN,NaN,NaN,NaN,...,False,False,False,NaN,NaN,NaN,NaN,NaN,0.000000,0.004484


AttributeError: 'tuple' object has no attribute 'display_mae_mfe_analysis'

In [96]:
report

(<finlab.core.report.Report at 0x17707b290>,)